### Libraries and source files

In [179]:

import xlwings as xw
import os
import pandas as pd
import numpy as np
pd.options.display.max_rows = None
pd.options.display.max_columns = None

In [180]:
# --  Choose which file to import from below
extended_df_name = 'extended_fmeca.xlsx' # The whole fmeca - which has all the newly calculated columns
req_calc_name = 'fmeca_req_calc.xlsx' # A slice of the fmeca with selected columns that require calculation. The order of columns is important here.
correct_abts_name = 'correct_abts.xlsx' # A slice of the fmeca which requires abt corrections.
correct_dim_name = 'correct_dim.xlsx' # A slice of fmeca which requires correct diminution levels

# -- File path to import from
file_location = 'C:/Users/nisha/OneDrive - Floating Solutions Consulting/Documents/04. Projects/JAD-01 MV Main Deck/03. Working/FMECA/Batch 5 FMECA 2022/FMECA spreadsheets/AN_working/ProcessedData'

In [181]:
df = pd.read_excel(os.path.join(file_location,req_calc_name),sheet_name='Sheet1')

In [182]:
df.head(1)

,Stiffener(s),stiff_anom_combo,stiffener_dim,h,grade,ps,Average UTM Reading (mm),tp,tw,tf,"Nearest transverse member (Fr., TWF,TBHD)",Longitudinal\n(l) (distance\nfrom in mm),Frame(s),Primary Structure,Final Anomaly description,Location,Detail
0,BP,BP_4SWBT-LBHD- CR-545,350x19.5 + 0x0 L2,0.0,AH36,Sec,14.4,9.85,NaN,NaN,TB,0,FR-47,Long'l bhd,Generalised corrosion,Tank boundary plating,Greater than 6mm remaining


### Data cleaning

Renaming parameters, harmonising structure names

In [183]:
def renameColumns():
    df.rename(columns={"Stiffener(s)":"stiffener","Nearest transverse member (Fr., TWF,TBHD)":"nearest_transverse","Longitudinal\n(l) (distance\nfrom in mm)":"distance_to_transverse","Frame(s)":"frame",
    "Final Anomaly description":"anomaly_type","Average UTM Reading (mm)":"Avg_utm","Primary Structure":"primary_structure","Location":"location",
    "Detail":"detail"},inplace=True)
    return None

renameColumns()

In [184]:
df["frame"].unique()

array(['FR-47', 'FR-46', 'FR-49', 'FR-48', 'TWF47-48', 'TWF54-55',
       'TWF48-49', 'TWF50-51', 'TWF54', 'TWF46-47', '49-50', '52-53'],
      dtype=object)

In [185]:
existing_map = ['FR-47', 'FR-46', 'FR-49', 'FR-48', 'TWF47-48', 'TWF54-55',
       'TWF48-49', 'TWF50-51', 'TWF54', 'TWF46-47', '49-50', '52-53']
new_map = ['FR-47', 'FR-46', 'FR-49', 'FR-48', 'FR-48', 'FR-55',
       'FR-49', 'FR-51', 'FR-54', 'FR-47', 'FR-50', 'FR-53']

map_dict = dict(zip(existing_map,new_map))

In [186]:
df["frame"] = df["frame"].map(map_dict)

In [187]:
df[df["nearest_transverse"]=='TB']

,stiffener,stiff_anom_combo,stiffener_dim,h,grade,ps,Avg_utm,tp,tw,tf,nearest_transverse,distance_to_transverse,frame,primary_structure,anomaly_type,location,detail
0,BP,BP_4SWBT-LBHD- CR-545,350x19.5 + 0x0 L2,0.00,AH36,Sec,14.400000,9.850000,NaN,NaN,TB,0,FR-47,Long'l bhd,Generalised corrosion,Tank boundary plating,Greater than 6mm remaining
176,LL23,LL23_4SWBT-LBHD- CR-450,500x10.5 + 150x23 L2,4.49,AH36,Sec,11.566667,9.520000,NaN,NaN,TB,100,FR-47,Long'l bhd,Generalised corrosion,Tank boundary plating,Greater than 6mm remaining
177,LL23,LL23_4SWBT-LBHD- CR-451,500x10.5 + 150x23 L2,4.49,AH36,Sec,12.400000,10.600000,NaN,NaN,TB,530,FR-47,Long'l bhd,Generalised corrosion,Tank boundary plating,Greater than 6mm remaining
178,LL23,LL23_4SWBT-LBHD- CR-452,500x10.5 + 150x23 L2,4.49,AH36,Sec,0.000000,9.166667,NaN,NaN,TB,210,FR-47,Long'l bhd,Generalised corrosion,Tank boundary plating,Less than 6mm remaining
179,LL23,LL23_4SWBT-LBHD- CR-453,500x10.5 + 150x23 L2,4.49,AH36,Sec,11.400000,10.500000,NaN,NaN,TB,0,FR-47,Long'l bhd,Generalised corrosion,Tank boundary plating,Greater than 6mm remaining
220,LL27,LL27_4SWBT-LBHD- CR-538,350x10 + 100x14 L2,1.00,AH36,Sec,6.500000,11.433333,NaN,NaN,TB,530,FR-47,Long'l bhd,Generalised corrosion,Stiffener web,Greater than 6mm remaining


#### Changing frames to the nearest defined frame in global loads calculation

Not all the frames are defined in the global loads calculation.<br>
Therefore, the rule that is being followed on this project is to change the frame to the nearest defined frame such that the bending moment of the frame can be applied in the calculations.
In the below script a problem specific solution has been implemented. A more general code can be written to replace it with the nearest defined Frame name.

In [188]:
# -- Existing cleaned frame names
a = ['FR-47', 'FR-46', 'FR-49', 'FR-48', 'FR-48', 'FR-55',
       'FR-49', 'FR-51', 'FR-54', 'FR-47', 'FR-50', 'FR-53'] 

# -- Nearest defined frames
b = ['46', '46', '51', '46', '46', '56',
       '51', '51', '56', '46', '51', '51']

map_nearest_frame = dict(zip(a,b))

In [189]:
df['frame_number'] = df['frame'].map(map_nearest_frame)

#### Horizontal location of the anomaly

In reference to the nearest anomlay.
The nearest anomaly column has inconsistencies.
The names in this column will be replaced with proper names.

In [190]:
df['nearest_transverse'].unique()

array(['TB', 'FR-47', 'FR-46', 'TBHD 46', 'FR-48', 'FR-49', 'TWF47',
       'TWF54', 'TWF48', 'TWF49', 'TWF51', 'Vert Stiffener', 'TWF46',
       'FR.49', 'FR.53', 'Vertical Stiffener', 'LBHD'], dtype=object)

In [191]:
old_trns = ['TB', 'FR-47', 'FR-46', 'TBHD 46', 'FR-48', 'FR-49', 'TWF47',
       'TWF54', 'TWF48', 'TWF49', 'TWF51', 'Vert Stiffener', 'TWF46',
       'FR.49', 'FR.53', 'Vertical Stiffener', 'LBHD']
new_trns = ['TB', 'FR-47', 'FR-46', 'FR-46', 'FR-48', 'FR-49', 'FR-47',
       'FR-54', 'FR-48', 'FR-49', 'FR-51', 'Vertical Stiffener', 'FR-46',
       'FR-49', 'FR-53', 'Vertical Stiffener', 'LBHD']

map_nearest_trns = dict(zip(old_trns,new_trns))

In [192]:
df['nearest_transverse'] = df['nearest_transverse'].map(map_nearest_trns)

### Data Munging

#### New column for span

In [193]:
def span(row,span1,span2):
    stiffener = row['stiffener']
    if stiffener == "LL26" or stiffener == "LL27" or stiffener == 'BP' or stiffener == "UD":
        return span2
    else:
        return span1



In [194]:
df['span'] = df.apply(span, args=(5095,2547.5),axis=1)

In [195]:
df.head(3)

,stiffener,stiff_anom_combo,stiffener_dim,h,grade,ps,Avg_utm,tp,tw,tf,nearest_transverse,distance_to_transverse,frame,primary_structure,anomaly_type,location,detail,frame_number,span
0,BP,BP_4SWBT-LBHD- CR-545,350x19.5 + 0x0 L2,0.000,AH36,Sec,14.40,9.85,NaN,NaN,TB,0,FR-47,Long'l bhd,Generalised corrosion,Tank boundary plating,Greater than 6mm remaining,46,2547.5
1,LL00,LL00_4SWBT-LBHD- CR-219,350x10 + 150x15.5 L2,23.497,AH36,Sec,6.99,6.50,NaN,NaN,FR-47,0,FR-46,Long'l bhd,Generalised corrosion,Stiffener web,Greater than 6mm remaining,46,5095.0
2,LL00,LL00_4SWBT-LBHD- CR-220,350x10 + 150x15.5 L2,23.497,AH36,Sec,6.60,7.60,NaN,NaN,FR-47,650,FR-46,Long'l bhd,Generalised corrosion,Stiffener web,Greater than 6mm remaining,46,5095.0


#### Anomaly position ratio

In [196]:
def positionRatio(row):
    position = row['distance_to_transverse']
    span = row['span']
    try:
        return np.round(position/span,2)
    except:
        return 'error'

In [197]:
df['anom_position_ratio'] = df.apply(positionRatio, axis=1)

In [198]:
df['anom_position_ratio'].unique()

array([0.0, 0.13, 0.4, 0.23, 0.27, 0.28, 0.39, 0.16, 0.04, 0.35, 0.47,
       0.25, 0.36, 0.03, 0.54, 0.41, 0.26, 0.37, 0.52, 0.18, 0.3, 0.07,
       0.1, 0.5, 0.14, 0.21, 0.22, 0.2, 0.19, 0.48, 0.11, 0.08, 0.43,
       0.02, 0.38, 0.06, 0.09, 0.6, 0.76, 0.7, 0.12, 0.44, 0.33, 0.34,
       3.21, 0.05, 0.17, 0.45, 'error', nan, 0.32, 0.29, 0.24, 0.01, 0.49,
       nan, 0.51, 0.58, 0.15, 0.61, 0.31], dtype=object)

#### Bending moment ratio calculation

In [205]:
def bmRatioFixedBeam(row):
    '''Bending moment ratio for a given x location along the stiffener
       Applicable only if the arrangement is Fixed end beam
       '''
    try:
        x = float(row['distance_to_transverse'])
        l = float(row['span'])
        if x != 0:
            try:
                x = float(row['distance_to_transverse'])
                l = float(row['span'])
                bmPerc = np.round(np.abs(((6*l*x)-(l**2)-(6*x**2))/(l**2)),2)
            except:
                bmPerc = 'error'
        else:
            bmPerc = 'anom located at the end'
    except:
        bmPerc = 'error'
    return bmPerc

In [206]:
def sfRatioFixedBeam(row):
    '''Shear force ratio for a given x location along the stiffener
       Applicable only if the arrangement is Fixed end beam
       '''
    try:
        x = float(row['distance_to_transverse'])
        l = float(row['span'])
        if x != 0:
            try:
                x = float(row['distance_to_transverse'])
                l = float(row['span'])
                sfPerc = np.round(np.abs(((l/2)-x)/(l/2)),2)
            except:
                sfPerc = 'error'
        else:
            sfPerc = 'anom located at the end'
    except:
        sfPerc = 'error'
    return sfPerc

In [207]:
df['bm%'] = df.apply(bmRatioFixedBeam,axis=1)

In [208]:
df['sf%'] = df.apply(sfRatioFixedBeam,axis=1)

In [209]:
df.head(4)

,stiffener,stiff_anom_combo,stiffener_dim,h,grade,ps,Avg_utm,tp,tw,tf,nearest_transverse,distance_to_transverse,frame,primary_structure,anomaly_type,location,detail,frame_number,span,anom_position_ratio,bm%,sf%
0,BP,BP_4SWBT-LBHD- CR-545,350x19.5 + 0x0 L2,0.000,AH36,Sec,14.40,9.85,NaN,NaN,TB,0,FR-47,Long'l bhd,Generalised corrosion,Tank boundary plating,Greater than 6mm remaining,46,2547.5,0.0,anom located at the end,anom located at the end
1,LL00,LL00_4SWBT-LBHD- CR-219,350x10 + 150x15.5 L2,23.497,AH36,Sec,6.99,6.50,NaN,NaN,FR-47,0,FR-46,Long'l bhd,Generalised corrosion,Stiffener web,Greater than 6mm remaining,46,5095.0,0.0,anom located at the end,anom located at the end
2,LL00,LL00_4SWBT-LBHD- CR-220,350x10 + 150x15.5 L2,23.497,AH36,Sec,6.60,7.60,NaN,NaN,FR-47,650,FR-46,Long'l bhd,Generalised corrosion,Stiffener web,Greater than 6mm remaining,46,5095.0,0.13,0.33,0.74
3,LL00,LL00_4SWBT-LBHD- CR-221,350x10 + 150x15.5 L2,23.497,AH36,Sec,6.30,6.40,NaN,NaN,FR-47,650,FR-46,Long'l bhd,Pit within generalised corrosion,Stiffener web,Greater than 6mm remaining,46,5095.0,0.13,0.33,0.74


In [167]:
xw.view(df)